In [1]:
import csv
from itertools import combinations
from collections import OrderedDict
from scipy.stats import chi2_contingency, fisher_exact
from statsmodels.sandbox.stats.multicomp import multipletests
from statsmodels.stats.proportion import proportion_confint

# Parse input table

In [2]:
microscopy = OrderedDict()
with open('microscopy_counts.tsv') as inf:
    next(inf)
    for row in csv.reader(inf, delimiter = '\t'):
        if row[1] not in microscopy:
            microscopy[row[1]] = OrderedDict()
        microscopy[row[1]][row[0]] = [int(x) for x in row[2:]]
microscopy

OrderedDict([('Andalusicin',
              OrderedDict([('1', [84, 72]),
                           ('2', [306, 277]),
                           ('3', [291, 300]),
                           ('4', [461, 420])])),
             ('CCCP',
              OrderedDict([('1', [3, 116]),
                           ('2', [9, 349]),
                           ('3', [13, 395])])),
             ('DMSO',
              OrderedDict([('1', [3, 140]),
                           ('2', [5, 128]),
                           ('3', [7, 564]),
                           ('4', [1, 297])])),
             ('Nisin',
              OrderedDict([('1', [137, 2]),
                           ('2', [202, 1]),
                           ('3', [486, 10])]))])

# Compare replicates

In [3]:
rep_combs = OrderedDict()
rep_fisher = OrderedDict()
for treatment in microscopy:
    rep_combs[treatment] = []
    rep_fisher[treatment] = []
    for i in combinations(microscopy[treatment], 2):
        rep_combs[treatment].append(i)
        rep_fisher[treatment].append(fisher_exact([microscopy[treatment][x] for x in i]))

In [4]:
rep_fisher

OrderedDict([('Andalusicin',
              [(1.0561002178649237, 0.7869767204779357),
               (1.2027491408934707, 0.3228790367648616),
               (1.0629067245119306, 0.7942176509664645),
               (1.1388589080352822, 0.26802988004568523),
               (1.006444943890616, 0.9574241030839763),
               (0.8837310195227766, 0.2640667251855691)]),
             ('CCCP',
              [(1.0028735632183907, 1.0),
               (0.7858090185676393, 1.0),
               (0.7835574167952392, 0.6673234846661388)]),
             ('DMSO',
              [(0.5485714285714286, 0.4879238881185358),
               (1.726530612244898, 0.42792798442693414),
               (6.364285714285714, 0.10221575071817324),
               (3.1473214285714284, 0.05714533360447075),
               (11.6015625, 0.011938263878003936),
               (3.6861702127659575, 0.2756818379628697)]),
             ('Nisin',
              [(0.33910891089108913, 0.5687550124233149),
               (1.40

In [7]:
pvals = []
for y in rep_fisher.values():
    pvals += [x[1] for x in y]

In [8]:
multipletests(pvals, alpha=0.05, method='bonferroni')

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False]),
 array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.21488875, 1.        ,
        1.        , 1.        , 1.        ]),
 0.002845571131556368,
 0.002777777777777778)

No significant difference between replicates of each treatment were found

# Pool replicates

In [9]:
pooled = OrderedDict()
for treatment in microscopy:
    pooled[treatment] = [0,0]
    for date in microscopy[treatment]:
        pooled[treatment][0] += microscopy[treatment][date][0]
        pooled[treatment][1] += microscopy[treatment][date][1]

In [10]:
pooled

OrderedDict([('Andalusicin', [1142, 1069]),
             ('CCCP', [25, 860]),
             ('DMSO', [16, 1129]),
             ('Nisin', [825, 13])])

# Calculate confidence intervals

In [11]:
ci = {x : proportion_confint(pooled[x][0], sum(pooled[x]), alpha=0.05, method='wilson') for x in pooled}

In [12]:
ci

{'Andalusicin': (0.49566790140495093, 0.5372915683165835),
 'CCCP': (0.019206150646389936, 0.04136872273106304),
 'DMSO': (0.008619462853019705, 0.02257845270400242),
 'Nisin': (0.973640205691301, 0.9909119572985116)}

In [13]:
ci_norm = {x : proportion_confint(pooled[x][0], sum(pooled[x]), alpha=0.05, method='normal') for x in pooled}

In [14]:
ci_norm

{'Andalusicin': (0.49567848180093804, 0.5373382527083338),
 'CCCP': (0.017332866869053512, 0.039164308272189426),
 'DMSO': (0.007174771226958878, 0.02077282702631623),
 'Nisin': (0.976119662651585, 0.9928540843651213)}

# Test difference between treatments

In [15]:
post_combs = []
post_fisher = []
for i in combinations(pooled, 2):
    post_combs.append(i)
    post_fisher.append(fisher_exact([pooled[x] for x in i]))

In [16]:
post_fisher

[(36.74911131898971, 5.519159220622407e-178),
 (75.38108044901777, 5.142711028702052e-239),
 (0.016833630977690847, 1.9964059799926474e-168),
 (2.051235465116279, 0.02604305303173919),
 (0.0004580690627202255, 0.0),
 (0.00022331373969992216, 0.0)]

In [17]:
post_pvals = [x[1] for x in post_fisher]

In [18]:
multipletests(post_pvals, alpha=0.05, method='bonferroni')

(array([ True,  True,  True, False,  True,  True]),
 array([3.31149553e-177, 3.08562662e-238, 1.19784359e-167, 1.56258318e-001,
        0.00000000e+000, 0.00000000e+000]),
 0.008512444610847103,
 0.008333333333333333)

In [19]:
post_combs

[('Andalusicin', 'CCCP'),
 ('Andalusicin', 'DMSO'),
 ('Andalusicin', 'Nisin'),
 ('CCCP', 'DMSO'),
 ('CCCP', 'Nisin'),
 ('DMSO', 'Nisin')]

Difference between DMSO and CCCP appeared to be non-significant